# Spark GraphFrames - Solutions

Before running present notebook please install the GraphFrames package into your cluster by following the steps in the pdf available with this notebook.  
To check you have everything setup correctly run the below code and make sure there are no errors. If you get a message about connecting to a cluster click *ok*.

In [0]:
#import the package we just installed
from graphframes import *
#import data types - All data types of Spark SQL are located in the package of pyspark.sql.types
from pyspark.sql.types import *
#row can be used to create a row object by using named arguments
from pyspark.sql import Row

<b>Graphs:</b> Graphs are mathematical structures used to model pairwise relations between objects. A graph in this context is made up of vertices (also called nodes or points) which are connected by edges (also called links or lines). A distinction is made between undirected graphs, where edges link two vertices symmetrically, and directed graphs, where edges link two vertices asymmetrically; see Graph (discrete mathematics) for more detailed definitions and for other variations in the types of graph that are commonly considered (follow this link: https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)). Graphs are one of the prime objects of study in discrete mathematics.

![image](https://mathinsight.org/media/image/image/small_undirected_network_labeled.png)

Graph processing is important aspect of analysis that applies to a lot of use cases. Fundamentally graph theory and processing are about defining relationships between different nodes and edges. Nodes or vertices are the units while edges define the relationships between nodes. This works great for social network analysis and running algorithms like PageRank to better understand and weigh relationships.

Some business use cases could be to look at central people in social networks [who is most popular in a group of friends], importance of papers in bibliographic networks [which papers are most referenced], and of course ranking web pages.

## PageRank

For this class we will look into the page rank algorithm again but – this time – from another point of view. As you might remenber, the PageRank of a particular web page indicates its relative importance within a group of web pages. The higher the page rank, the higher up it will appear in a search result list. The importance of a page is defined by the importance of all the web pages that provide an outbound link to the web page in consideration.
 For example, say that web page X has very high relative importance. Web page X is outbounding to web page Y (webpage X links to Y); hence, web page Y will also have high importance. However, if webpage X links to many webpages, including Y, the importance of Y is lower.

For more information on pagerank you either:

Follow through the worked example: https://www.cs.princeton.edu/~chazelle/courses/BIB/pagerank.htm

Take a look at one of the many youtube explanations, for example (10min): https://www.youtube.com/watch?v=qpIqLjfcabs

Part of what is important here is to understand the data format we use to represent a graph. I have commented the steps below:

In [0]:
# First we start with the list of vertices, I have 5 vertices here, each named with a letter of the alphabet
verticesList = ['a', 'b', 'c', 'd']
# We load this list into an RDD as we have seen before
verticesListRDD = sc.parallelize(verticesList, 1)
# We convert this RDD to a dataframe using row objects and specifying a schema.
verticesListRowsRDD = verticesListRDD.map( lambda data : Row(data))
verticesSchema = StructType([StructField('id', StringType(), True)])
verticesDataFrame = sqlContext.createDataFrame(verticesListRowsRDD, verticesSchema).persist()
#Here I show the schema of the Dataframe.
verticesDataFrame.show()

+---+
| id|
+---+
|  a|
|  b|
|  c|
|  d|
+---+



Next we need to know how the vertices are connected, a list of edges. The process is very similar to that we just saw.

First, we have to create a list of edges. Each edge in the list will be defined by a tuple.

Then, we have to create an RDD of edges.

Thereafter, we have to transform our RDD to an RDD of row objects. This will be followed by creating a schema and a DataFrame of edges. Let’s perform the steps:

In [0]:
edgeDataList = [('a','b'), ('a','c'), ('a','d'), ('b', 'c'),
('b', 'd'),('c', 'b'), ('d', 'a'), ('d', 'c')]
sourceColumn = StructField('src', StringType(),True)
destinationColumn = StructField('dst', StringType(), True)
edgeSchema = StructType([sourceColumn, destinationColumn])
edgeRDD = sc.parallelize(edgeDataList, 1)
edgeRDDRows = edgeRDD.map( lambda data : Row(data[0], data[1]))
edgeDataFrame = sqlContext.createDataFrame(edgeRDDRows, edgeSchema).persist()
edgeDataFrame.show()

+---+---+
|src|dst|
+---+---+
|  a|  b|
|  a|  c|
|  a|  d|
|  b|  c|
|  b|  d|
|  c|  b|
|  d|  a|
|  d|  c|
+---+---+



In [0]:
edgeRDD.take(10)

Out[4]: [('a', 'b'),
 ('a', 'c'),
 ('a', 'd'),
 ('b', 'c'),
 ('b', 'd'),
 ('c', 'b'),
 ('d', 'a'),
 ('d', 'c')]

In [0]:
edgeRDDRows.take(10)

Out[5]: [<Row('a', 'b')>,
 <Row('a', 'c')>,
 <Row('a', 'd')>,
 <Row('b', 'c')>,
 <Row('b', 'd')>,
 <Row('c', 'b')>,
 <Row('d', 'a')>,
 <Row('d', 'c')>]

At this moment, we have `verticesDataFrame`, a DataFrame of vertices; and `edgeDataFrame`, a DataFrames of edges. Using these two, we can create our graph. The GraphFrame Python class is defined under the `graphframes.graphframe` submodule. GraphFrame() takes the vertices and edges DataFrames and returns a GraphFrames object. We have our `GraphFrames` object, `ourGraph`. We can fetch all the vertices as follows: In `GraphFrames`, we can create a graph by using the following code lines:

**If you get an error on the below cell make sure you installed the correct version of graphframes**

In [0]:
from graphframes import *

ourGraph = GraphFrame(verticesDataFrame, edgeDataFrame)
ourGraph.vertices.show()
ourGraph.edges.show()

/databricks/spark/python/pyspark/sql/dataframe.py:170: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+---+
| id|
+---+
|  a|
|  b|
|  c|
|  d|
+---+

+---+---+
|src|dst|
+---+---+
|  a|  b|
|  a|  c|
|  a|  d|
|  b|  c|
|  b|  d|
|  c|  b|
|  d|  a|
|  d|  c|
+---+---+



Page rank for pages can be found by using the `pageRank()` function, which is defined on the GraphFrames object.

`pageRank()` function take two arguments: first is resetProbability (is the alpha explained below) and the second is maxIter

Informally Page Rank Explanation: consider someone browsing the web ("surfer"). At every page, the surfer either follows a link on the page to another page or does something else. When following a link, without any other information, the surfer picks a link from the page at random and follows that one. When “doing something else,” the surfer moves to a random page on the web and restarts the surfing. To generate a simple model, we assume both of these behaviors even though they may seem ridiculous. When stated mathematically, this random surfer model is called a Markov chain because the behavior of the surfer only depends upon the current page and not the history of previous pages. Let α be the probability that the surfer follows a link; then 1 − α is the probability that the surfer “does something else.” 

Now imagine that we let the surfer run for a long time. The PageRank of a page is the probability of finding the surfer at that page as the surfing time becomes infinite. A key assumption behind PageRank is that pages where we are more likely to find the random surfer are more important pages and thus we can view the PageRank as a measure of the page’s importance.

*Source: https://web.stanford.edu/group/SOL/dissertations/pagerank-sensitivity-thesis-online.pdf*

In [0]:
pageRanks = ourGraph.pageRank(resetProbability=0.15, maxIter = 5)

/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [0]:
pageRanks.vertices.show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|  a|0.5169666927083334|
|  d|0.8798845717592593|
|  b|1.3562974710648146|
|  c|1.2468512644675924|
+---+------------------+



## More graphframes applications

Let's look at a slightly more interesting dataset. Look at the code below, you should see we are loading a number of people as vertices, then they can either connect as friends (bi-directional) or one can follow another (uni directional).

In [0]:

vertices = spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])
edges = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('1', '4', 'follows'),
                               ('4', '5', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['src', 'dst', 'type'])
g = GraphFrame(vertices, edges)
## Take a look at the DataFrames
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

+---+------+---------+---+
| id|  name|firstname|age|
+---+------+---------+---+
|  1|Carter|  Derrick| 50|
|  2|   May|  Derrick| 26|
|  3| Mills|     Jeff| 80|
|  4|  Hood|   Robert| 65|
|  5| Banks|     Mike| 93|
| 98|  Berg|      Tim| 28|
| 99|  Page|    Allan| 16|
+---+------+---------+---+

+---+---+-------+
|src|dst|   type|
+---+---+-------+
|  1|  2| friend|
|  2|  1| friend|
|  3|  1| friend|
|  1|  3| friend|
|  2|  3|follows|
|  1|  4|follows|
|  4|  5|follows|
|  3|  4| friend|
|  4|  3| friend|
|  5|  3| friend|
|  3|  5| friend|
|  4|  5|follows|
| 98| 99| friend|
| 99| 98| friend|
+---+---+-------+

+---+------+
| id|degree|
+---+------+
|  1|     5|
|  2|     3|
|  3|     7|
|  5|     4|
|  4|     5|
| 98|     2|
| 99|     2|
+---+------+



Undirected graphs have edges that do not have a direction. The edges indicate a two-way relationship, in that each edge can be traversed in both directions. If your DataFrame only consist of two-way directed edges, you may be interested in analyzing undirected edges. You can convert your graph by mapping a function over the edges DataFrame that deletes the row if src ≥ dst (or the other way around). Then each edge is only included once.

Start from the dataframe called edges, notice that directional edges are listed [a,b, follows] where a < b (so the lowest number edge is listed first), unidirectional edges are listed twice. Create a new dataframe edgesUnidirectional that only contains edges in one direction but includes both the previously unidirectional friends links and the directional follows links.

In [0]:
edgesUnidirectional = edges.where(edges.src < edges.dst)
edgesUnidirectional.show()

+---+---+-------+
|src|dst|   type|
+---+---+-------+
|  1|  2| friend|
|  1|  3| friend|
|  2|  3|follows|
|  1|  4|follows|
|  4|  5|follows|
|  3|  4| friend|
|  3|  5| friend|
|  4|  5|follows|
| 98| 99| friend|
+---+---+-------+



We can filter edges and vertices directly as well. If we want to work with a GraphFrame with only follows and people over 30 for example we can use a subgraph, for example:

In [0]:
g.filterEdges("type = 'follows'").filterVertices("age > 30").edges.show()

+---+---+-------+
|src|dst|   type|
+---+---+-------+
|  1|  4|follows|
|  4|  5|follows|
|  4|  5|follows|
+---+---+-------+



<b> Degrees </b> One question we might be interested in is how many inbound and outbound connections each vertices has, we can use the inDegrees and outDegrees functions to calculate this:

In [0]:
g.inDegrees.show()
g.outDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  2|       1|
|  1|       2|
|  3|       4|
|  5|       3|
|  4|       2|
| 98|       1|
| 99|       1|
+---+--------+

+---+---------+
| id|outDegree|
+---+---------+
|  1|        3|
|  3|        3|
|  2|        2|
|  4|        3|
|  5|        1|
| 98|        1|
| 99|        1|
+---+---------+



<b>Connected components</b> A connected component of a graph is a subgraph in which any two vertices are connected to each other by one or more edges, and which is connected to no additional vertices in the supergraph. In the graph below there would be 3 connected component subgraphs:

![image](https://www.google.com/imgres?imgurl=https%3A%2F%2Ftutorialspoint.dev%2Fimage%2FSCC.png&imgrefurl=https%3A%2F%2Ftutorialspoint.dev%2Fdata-structure%2Fgraph-data-structure%2Fstrongly-connected-components&tbnid=5e31TFF3uPb4IM&vet=12ahUKEwivjuSxlKX2AhVn_bsIHYNiBlIQMygBegQIARAe..i&docid=WBvcXT3tPVeF_M&w=757&h=424&q=https%3A%2F%2Ftutorialspoint.dev%2Fimage%2Fconnectedcomponents.png&hl=en&ved=2ahUKEwivjuSxlKX2AhVn_bsIHYNiBlIQMygBegQIARAe)



Graphframes can compute the connected component membership of each vertex and return a DataFrame with each vertex assigned a component ID. The GraphFrames connected components implementation can take advantage of checkpointing to improve performance. This will take up tp 10 min to run.

In [0]:
# Note that this takes some time to run
# The resulting dataframe adds a "component" column to the dataset
sc.setCheckpointDir("/tmp/graphframes-example-connected-components")
g.connectedComponents().show()

+---+------+---------+---+------------+
| id|  name|firstname|age|   component|
+---+------+---------+---+------------+
|  1|Carter|  Derrick| 50|154618822656|
|  2|   May|  Derrick| 26|154618822656|
|  3| Mills|     Jeff| 80|154618822656|
|  4|  Hood|   Robert| 65|154618822656|
|  5| Banks|     Mike| 93|154618822656|
| 98|  Berg|      Tim| 28|317827579904|
| 99|  Page|    Allan| 16|317827579904|
+---+------+---------+---+------------+



What does the result say about how are people connected in our graph? Take a look at who is connected to who in the data and see if you can match a pattern to what is being reported (the component number is not important, only which nodes have the same component).

<b> Shortest path </b> Graphframes includes a simple shortest path algorithm, it finds the distance between all nodes and a given set of landmarks.

In the example below we will see from each node how many steps does it take to get to a landmark. So for example [5 -> 1, 1 -> 1] would means that to get to vertice named 5 takes 1 edge, to get to a vertice named 1 also takes 1 step.

In [0]:
g.shortestPaths(landmarks=["1", "5"]).show()

+---+------+---------+---+----------------+
| id|  name|firstname|age|       distances|
+---+------+---------+---+----------------+
|  3| Mills|     Jeff| 80|{5 -> 1, 1 -> 1}|
| 98|  Berg|      Tim| 28|              {}|
| 99|  Page|    Allan| 16|              {}|
|  5| Banks|     Mike| 93|{5 -> 0, 1 -> 2}|
|  1|Carter|  Derrick| 50|{5 -> 2, 1 -> 0}|
|  4|  Hood|   Robert| 65|{5 -> 1, 1 -> 2}|
|  2|   May|  Derrick| 26|{5 -> 2, 1 -> 1}|
+---+------+---------+---+----------------+



<b>Search</b> Breadth-first search (BFS) is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key'[1]), and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

It uses the opposite strategy as depth-first search, which instead explores the highest-depth nodes first before being forced to backtrack and expand shallower nodes.

For more information on these alogrithms: https://en.wikipedia.org/wiki/Breadth-first_search

Take a look at the examples below.

In [0]:
g.bfs(fromExpr="id='1'", toExpr = "id='5'").show(10,False)

+------------------------+---------------+---------------------+---------------+--------------------+
|from                    |e0             |v1                   |e1             |to                  |
+------------------------+---------------+---------------------+---------------+--------------------+
|{1, Carter, Derrick, 50}|{1, 3, friend} |{3, Mills, Jeff, 80} |{3, 5, friend} |{5, Banks, Mike, 93}|
|{1, Carter, Derrick, 50}|{1, 4, follows}|{4, Hood, Robert, 65}|{4, 5, follows}|{5, Banks, Mike, 93}|
|{1, Carter, Derrick, 50}|{1, 4, follows}|{4, Hood, Robert, 65}|{4, 5, follows}|{5, Banks, Mike, 93}|
+------------------------+---------------+---------------------+---------------+--------------------+



In [0]:
filteredPaths = g.bfs(
  fromExpr = "id = 1",
  toExpr = "age >= 65",
  edgeFilter = "type != 'friend'",
  maxPathLength = 4).dropDuplicates()
display(filteredPaths)

from,e0,to
"List(1, Carter, Derrick, 50)","List(1, 4, follows)","List(4, Hood, Robert, 65)"


<b> Motif </b> All networks, including biological networks, social networks, technological networks (e.g., computer networks and electrical circuits) and more, can be represented as graphs, which include a wide variety of subgraphs. One important local property of networks are so-called network motifs, which are defined as recurrent and statistically significant sub-graphs or patterns.

Network motifs are sub-graphs that repeat themselves in a specific network or even among various networks. Each of these sub-graphs, defined by a particular pattern of interactions between vertices, may reflect a framework in which particular functions are achieved efficiently. Indeed, motifs are of notable importance largely because they may reflect functional properties. They have recently gathered much attention as a useful concept to uncover structural design principles of complex networks. Although network motifs may provide a deep insight into the network’s functional abilities, their detection is computationally challenging.

GraphFrame motif finding uses a simple Domain-Specific Language (DSL) for expressing structural queries. For example, graph.find("(a)-[e]->(b); (b)-[e2]->(a)") will search for pairs of vertices a,b connected by edges in both directions. It will return a DataFrame of all such structures in the graph, with columns for each of the named elements (vertices or edges) in the motif. In this case, the returned columns will be “a, b, e, e2.”

DSL for expressing structural patterns:

The basic unit of a pattern is an edge. For example, "(a)-[e]->(b)" expresses an edge e from vertex a to vertex b. Note that vertices are denoted by parentheses (a), while edges are denoted by square brackets [e].

A pattern is expressed as a union of edges. Edge patterns can be joined with semicolons. Motif "(a)-[e]->(b); (b)-[e2]->(c)" specifies two edges from a to b to c.

Within a pattern, names can be assigned to vertices and edges. For example, "(a)-[e]->(b)" has three named elements: vertices a,b and edge e. These names serve two purposes:

*  The names can identify common elements among edges. For example, "(a)-[e]->(b); (b)-[e2]->(c)" specifies that the same vertex b is the destination of edge e and source of edge e2.

*  The names are used as column names in the result DataFrame. If a motif contains named vertex a, then the result DataFrame will contain a column “a” which is a StructType with sub-fields equivalent to the schema (columns) of GraphFrame.vertices. Similarly, an edge e in a motif will produce a column “e” in the result DataFrame with sub-fields equivalent to the schema (columns) of GraphFrame.edges.

*  Be aware that names do not identify distinct elements: two elements with different names may refer to the same graph element. For example, in the motif "(a)-[e]->(b); (b)-[e2]->(c)", the names a and c could refer to the same vertex. To restrict named elements to be distinct vertices or edges, use post-hoc filters such as resultDataframe.filter("a.id != c.id").

It is acceptable to omit names for vertices or edges in motifs when not needed. E.g., "(a)-[]->(b)" expresses an edge between vertices a,b but does not assign a name to the edge. There will be no column for the anonymous edge in the result DataFrame. Similarly, "(a)-[e]->()" indicates an out-edge of vertex a but does not name the destination vertex. These are called anonymous vertices and edges.

An edge can be negated to indicate that the edge should not be present in the graph. E.g., "(a)-[]->(b); !(b)-[]->(a)" finds edges from a to b for which there is no edge from b to a.

For more information: http://graphframes.github.io/graphframes/docs/_site/user-guide.html#motif-finding

A worked example is available here with a few motif patterns applied. They use Scala as opposed to Python but you can ignore the code outside of the patterns they use: http://www.twesdai.com/2017/09/15/motif-analysis-using-apache-spark-graphframes/

Let's look at a simple example to try to make how this works clear. Imagine we might want to find all people that have a common friend with person 1, but are not friends with person one. First we find all common friends. Take a look at the find method, we look for where a is connected to b, b connected to c, but a not connected to c. In the next step we filter for results with person 1.

In [0]:
motifs = g.find("(a)-[]->(b); (b)-[]->(c); !(a)-[]->(c) ").dropDuplicates()

In [0]:
motifs.show()

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|{4, Hood, Robert,...|{3, Mills, Jeff, 80}|{1, Carter, Derri...|
|{5, Banks, Mike, 93}|{3, Mills, Jeff, 80}|{1, Carter, Derri...|
|{1, Carter, Derri...|{2, May, Derrick,...|{1, Carter, Derri...|
|{1, Carter, Derri...|{3, Mills, Jeff, 80}|{1, Carter, Derri...|
|{3, Mills, Jeff, 80}|{1, Carter, Derri...|{2, May, Derrick,...|
|{2, May, Derrick,...|{1, Carter, Derri...|{2, May, Derrick,...|
|{3, Mills, Jeff, 80}|{1, Carter, Derri...|{3, Mills, Jeff, 80}|
|{3, Mills, Jeff, 80}|{5, Banks, Mike, 93}|{3, Mills, Jeff, 80}|
|{3, Mills, Jeff, 80}|{4, Hood, Robert,...|{3, Mills, Jeff, 80}|
|{4, Hood, Robert,...|{3, Mills, Jeff, 80}|{4, Hood, Robert,...|
|{2, May, Derrick,...|{3, Mills, Jeff, 80}|{4, Hood, Robert,...|
|{5, Banks, Mike, 93}|{3, Mills, Jeff, 80}|{4, Hood, Robert,...|
|{2, May, Derrick,...|{1,

In [0]:
filteredMotif = motifs.filter("a.id = 1")

In [0]:
filteredMotif.show()

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|{1, Carter, Derri...|{2, May, Derrick,...|{1, Carter, Derri...|
|{1, Carter, Derri...|{3, Mills, Jeff, 80}|{1, Carter, Derri...|
|{1, Carter, Derri...|{4, Hood, Robert,...|{5, Banks, Mike, 93}|
|{1, Carter, Derri...|{3, Mills, Jeff, 80}|{5, Banks, Mike, 93}|
+--------------------+--------------------+--------------------+



## Bonus Exercise: Bikeshare

Here is your chance to apply what you have learned to some real data. We will be using a bike sharing dataset, but a very different one from last class

Here you’re going to be working with publicly available bike data from the Bay Area Bike Share portal, specifically analyzing the 2017 year of data. First we will download the data and read it into spark, we will also rename some of the columns to meet the requirements of graphframes.

In [0]:
%sh wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_trip_data.csv

--2024-05-10 11:04:24--  https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_trip_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43012650 (41M) [text/plain]
Saving to: ‘201508_trip_data.csv’

     0K .......... .......... .......... .......... ..........  0% 7.52M 5s
    50K .......... .......... .......... .......... ..........  0% 4.98M 7s
   100K .......... .......... .......... .......... ..........  0% 3.80M 8s
   150K .......... .......... .......... .......... ..........  0% 50.0M 6s
   200K .......... .......... .......... .......... ..........  0% 4.68M 7s
   250K .......... .......... .......... .......... ..........  0% 14.7M 6s
   300K .......... .......... .......... .......... ..........  0% 35.7

In [0]:
%sh wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_station_data.csv

--2024-05-10 11:04:28--  https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_station_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5272 (5.1K) [text/plain]
Saving to: ‘201508_station_data.csv’

     0K .....                                                 100% 32.4M=0s

2024-05-10 11:04:28 (32.4 MB/s) - ‘201508_station_data.csv’ saved [5272/5272]



In [0]:
tripDF = spark.read.csv('file:/databricks/driver/201508_trip_data.csv',header=True, inferSchema = True)
tripDF.printSchema()

root
 |-- Trip ID: integer (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Start Station: string (nullable = true)
 |-- Start Terminal: integer (nullable = true)
 |-- End Date: string (nullable = true)
 |-- End Station: string (nullable = true)
 |-- End Terminal: integer (nullable = true)
 |-- Bike #: integer (nullable = true)
 |-- Subscriber Type: string (nullable = true)
 |-- Zip Code: string (nullable = true)



In [0]:
stationDF = spark.read.csv('file:/databricks/driver/201508_station_data.csv',header=True, inferSchema = True)
stationDF.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: date (nullable = true)



In [0]:
stationVertices = stationDF.withColumnRenamed("name", "id").distinct()
tripEdges = tripDF.withColumnRenamed("Start Station", "src").withColumnRenamed("End Station", "dst")

In [0]:
tripEdges.show(10, False)

+-------+--------+---------------+---------------------------------------------+--------------+---------------+---------------------------------------------+------------+------+---------------+--------+
|Trip ID|Duration|Start Date     |src                                          |Start Terminal|End Date       |dst                                          |End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+---------------------------------------------+--------------+---------------+---------------------------------------------+------------+------+---------------+--------+
|913460 |765     |8/31/2015 23:26|Harry Bridges Plaza (Ferry Building)         |50            |8/31/2015 23:39|San Francisco Caltrain (Townsend at 4th)     |70          |288   |Subscriber     |2139    |
|913459 |1036    |8/31/2015 23:11|San Antonio Shopping Center                  |31            |8/31/2015 23:28|Mountain View City Hall                      |27          |35    |Subscriber 

In [0]:
stationVertices.show(10, False)

+----------+---------------------------------+---------+-----------+---------+--------+------------+
|station_id|id                               |lat      |long       |dockcount|landmark|installation|
+----------+---------------------------------+---------+-----------+---------+--------+------------+
|10        |San Jose City Hall               |37.337391|-121.886995|15       |San Jose|2013-08-06  |
|12        |SJSU 4th at San Carlos           |37.332808|-121.883891|19       |San Jose|2013-08-07  |
|3         |San Jose Civic Center            |37.330698|-121.888979|15       |San Jose|2013-08-05  |
|7         |Paseo de San Antonio             |37.333798|-121.886943|15       |San Jose|2013-08-07  |
|4         |Santa Clara at Almaden           |37.333988|-121.894902|11       |San Jose|2013-08-06  |
|5         |Adobe on Almaden                 |37.331415|-121.8932  |19       |San Jose|2013-08-05  |
|2         |San Jose Diridon Caltrain Station|37.329732|-121.901782|27       |San Jose|2013

Alright you now have the data formatted as you need to perform analysis. Take a look to make sure you understand what the data is. Basically you have bike stations (verticies) and trips between the stations (edges). 

Hint: in the cell above I used 'False' in the `show()`, this will be useful for you to prevent truncating station names. The highest ranked station should be San Jose Diridon Caltrain Station.

#####Create a graphframe and perform a pagerank to find the most import stations (make sure to set the maxIterations to 5).

In [0]:
#take the vertices and edges DataFrames and returns a GraphFrames object.
gbs = GraphFrame(stationVertices, tripEdges)

In [0]:
#show the edges
gbs.edges.show(10, False)

+-------+--------+---------------+---------------------------------------------+--------------+---------------+---------------------------------------------+------------+------+---------------+--------+
|Trip ID|Duration|Start Date     |src                                          |Start Terminal|End Date       |dst                                          |End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+---------------------------------------------+--------------+---------------+---------------------------------------------+------------+------+---------------+--------+
|913460 |765     |8/31/2015 23:26|Harry Bridges Plaza (Ferry Building)         |50            |8/31/2015 23:39|San Francisco Caltrain (Townsend at 4th)     |70          |288   |Subscriber     |2139    |
|913459 |1036    |8/31/2015 23:11|San Antonio Shopping Center                  |31            |8/31/2015 23:28|Mountain View City Hall                      |27          |35    |Subscriber 

In [0]:
#Run the pageRank()
pageRanks = gbs.pageRank(resetProbability=0.15, maxIter = 5)

In [0]:
#Show() the vertices, ranked
pageRanks.vertices.orderBy(pageRanks.vertices.pagerank,ascending=False).show(10,False)

+----------+----------------------------------------+---------+-----------+---------+-------------+------------+------------------+
|station_id|id                                      |lat      |long       |dockcount|landmark     |installation|pagerank          |
+----------+----------------------------------------+---------+-----------+---------+-------------+------------+------------------+
|2         |San Jose Diridon Caltrain Station       |37.329732|-121.901782|27       |San Jose     |2013-08-06  |4.084086216116643 |
|70        |San Francisco Caltrain (Townsend at 4th)|37.776617|-122.39526 |19       |San Francisco|2013-08-23  |3.3515435057991234|
|28        |Mountain View Caltrain Station          |37.394358|-122.076713|23       |Mountain View|2013-08-15  |2.5531183688195793|
|22        |Redwood City Caltrain Station           |37.486078|-122.232089|25       |Redwood City |2013-08-15  |2.4734490963466937|
|69        |San Francisco Caltrain 2 (330 Townsend) |37.7766  |-122.39547 |2

One question is: what are the most common trip paths? You can do this by performing a grouping operator and adding the edge counts together, think of how we used sparkSQL to implement SQL like queries last week. Remember the list of edges is just a dataframe so you can use SQL queries to find a result like this. In case you have forgotten this link has some information on groupBy:

https://sparkbyexamples.com/spark/using-groupby-on-dataframe/

The most common trip should be:

San Francisco Caltrain 2 (330 Townsend)  ->    Townsend at 7th.

In [0]:
tripEdgesSum = tripEdges.groupBy(tripEdges.src, tripEdges.dst).count()
tripEdgesSum.orderBy("count",ascending=False).show(10, False)

+---------------------------------------------+----------------------------------------+-----+
|src                                          |dst                                     |count|
+---------------------------------------------+----------------------------------------+-----+
|San Francisco Caltrain 2 (330 Townsend)      |Townsend at 7th                         |3748 |
|Harry Bridges Plaza (Ferry Building)         |Embarcadero at Sansome                  |3145 |
|2nd at Townsend                              |Harry Bridges Plaza (Ferry Building)    |2973 |
|Townsend at 7th                              |San Francisco Caltrain 2 (330 Townsend) |2734 |
|Harry Bridges Plaza (Ferry Building)         |2nd at Townsend                         |2640 |
|Embarcadero at Folsom                        |San Francisco Caltrain (Townsend at 4th)|2439 |
|Steuart at Market                            |2nd at Townsend                         |2356 |
|Embarcadero at Sansome                       |Ste

Create a new graphframe with only unique edges and check if all stations are connected (i.e. whether there are any stations without trips)

In [0]:
#Create tripEdgesUnique graphframe
tripEdgesUnique = tripEdges.dropDuplicates(["src", "dst"])

In [0]:
# We compute a union, all the edges (regardless of in/out) in the edges DataFrame
all_stations_with_edges = tripEdges \
.selectExpr("`Trip ID`", 'explode(array(src, dst)) AS src_dst') \
.dropDuplicates()

all_stations_with_edges.show()

+-------+--------------------+
|Trip ID|             src_dst|
+-------+--------------------+
| 913460|San Francisco Cal...|
| 913448|South Van Ness at...|
| 913451|Embarcadero at Fo...|
| 913455|      Post at Kearny|
| 913449|     Beale at Market|
| 913443|   Market at Sansome|
| 913459|San Antonio Shopp...|
| 913452|Yerba Buena Cente...|
| 913460|Harry Bridges Pla...|
| 913452|San Francisco Cal...|
| 913450|Embarcadero at Sa...|
| 913449|Temporary Transba...|
| 913459|Mountain View Cit...|
| 913453|Embarcadero at Fo...|
| 913448|      Post at Kearny|
| 913443|Embarcadero at Sa...|
| 913453|Embarcadero at Sa...|
| 913455|   2nd at South Park|
| 913451|Embarcadero at Sa...|
| 913450|   Steuart at Market|
+-------+--------------------+
only showing top 20 rows



In [0]:
# We find the set difference between all the vertices in the edges DataFrame and the vertices
stations_with_no_trips = stationVertices.select("id").subtract(all_stations_with_edges.select("src_dst")).collect()

In [0]:
# We extract the "id" from each filtered row
stations_with_no_trips_ls = list(map(lambda x: x["id"], stations_with_no_trips))

In [0]:
# Finally, we filter for all stations that are present in both datasets
from pyspark.sql.functions import col
connected_stations = stationVertices.filter(~col("id").isin(stations_with_no_trips_ls))

In [0]:
gbsUnique = GraphFrame(connected_stations, tripEdgesUnique)

In [0]:
tripEdgesUnique.select("src", "dst").show()

+--------------------+--------------------+
|                 src|                 dst|
+--------------------+--------------------+
|     Beale at Market|Temporary Transba...|
|Embarcadero at Sa...|   Steuart at Market|
|      Post at Kearny|   2nd at South Park|
|Harry Bridges Pla...|San Francisco Cal...|
|Temporary Transba...|San Francisco Cal...|
|     Spear at Folsom|San Francisco Cal...|
|      Post at Kearny|South Van Ness at...|
|   Market at Sansome|Broadway St at Ba...|
|San Antonio Shopp...|Mountain View Cit...|
|      Market at 10th|San Francisco Cal...|
|San Francisco Cal...|       Market at 4th|
|Mountain View Cal...|Castro Street and...|
|University and Em...|Cowper at University|
|  San Jose City Hall| San Salvador at 1st|
|       Market at 4th|Grant Avenue at C...|
|Embarcadero at Sa...|   Market at Sansome|
|Temporary Transba...|Grant Avenue at C...|
|Yerba Buena Cente...|San Francisco Cal...|
|San Francisco Cal...|Broadway St at Ba...|
|San Francisco Cal...|     Towns

In [0]:

gbsUnique = GraphFrame(stationVertices, tripEdgesSum)

In [0]:
#Computes the connected component membership of each vertex and returns a graph with each vertex assigned a component ID.
cc = gbsUnique.connectedComponents()

In [0]:
#show the graphframe with component
cc.select("id","component").orderBy("component").show()

+--------------------+---------+
|                  id|component|
+--------------------+---------+
|Cowper at University|        0|
|  San Jose City Hall|        0|
|California Ave Ca...|        0|
|South Van Ness at...|        0|
|   Market at Sansome|        0|
|SJSU 4th at San C...|        0|
|     Townsend at 7th|        0|
|Castro Street and...|        0|
|Embarcadero at Fo...|        0|
|Embarcadero at Sa...|        0|
|         Ryland Park|        0|
|Mechanics Plaza (...|        0|
|San Antonio Shopp...|        0|
|          Mezes Park|        0|
|SJSU - San Salvad...|        0|
|Harry Bridges Pla...|        0|
|Embarcadero at Br...|        0|
|       5th at Howard|        0|
|Commercial at Mon...|        0|
|Temporary Transba...|        0|
+--------------------+---------+
only showing top 20 rows



In [0]:
#It seems a lot of stations are in component 0. Try to filter where component > 0
cc.where(cc.component > 0).show(10)

+----------+--------------------+---------+-----------+---------+-------------+------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|   component|
+----------+--------------------+---------+-----------+---------+-------------+------------+------------+
|        47|     Post at Kearney|37.788975|-122.403452|       19|San Francisco|  2013-08-19|317827579904|
|        46|Washington at Kea...|37.795425|-122.404767|       15|San Francisco|  2013-08-19| 17179869184|
+----------+--------------------+---------+-----------+---------+-------------+------------+------------+



Remember that in this instance you’ve got a directed graph. That means that your trips are directional - from one location to another. Therefore you get access to a wealth of analysis that you can use. You can find the number of trips that go into a specific station and leave from a specific station.

One interesting question you could ask is what is the station with the highest ratio of in degrees to out degrees. As in, what station acts as a pure trip sink. A station where trips end at but rarely start from. This station will end up with a lot of excess bikes that will need to be re-distributed.

In [0]:
gbsOut = gbs.outDegrees
gbsIn = gbs.inDegrees
# Note you may find it easier to use q SQL query here if that is what you are used to
gbsJoin = gbsIn.join(gbsOut, ["id"])
gbsDeg = gbsJoin.withColumn("Ratio", col("inDegree") / col("outDegree"))

In [0]:
gbsJoin.printSchema()

root
 |-- id: string (nullable = true)
 |-- inDegree: integer (nullable = false)
 |-- outDegree: integer (nullable = false)



In [0]:
gbsDeg.orderBy("Ratio",ascending=False).show(10)

+--------------------+--------+---------+------------------+
|                  id|inDegree|outDegree|             Ratio|
+--------------------+--------+---------+------------------+
|Redwood City Medi...|     230|      150|1.5333333333333334|
|San Mateo County ...|     187|      127|1.4724409448818898|
|SJSU 4th at San C...|     647|      475|1.3621052631578947|
|San Francisco Cal...|   34810|    26304|1.3233728710462287|
|Washington at Kearny|    3481|     2660|1.3086466165413533|
|Paseo de San Antonio|    1073|      856|1.2535046728971964|
|California Ave Ca...|     496|      400|              1.24|
|   Franklin at Maple|     100|       81|1.2345679012345678|
|Embarcadero at Va...|    6146|     5037|1.2201707365495336|
|   Market at Sansome|   13916|    11431|1.2173913043478262|
+--------------------+--------+---------+------------------+
only showing top 10 rows



Consider the station with the highest ratio of intrips to outtrips, find all stations connected by 1 or 2 trips to this station using motifs.

In [0]:
#search for pairs of vertices a,b connected by edges in both directions
motifsBS = gbsUnique.find("(a)-[]->(b); (b)-[]->(c)").dropDuplicates()

In [0]:
# Find stations connected by two trips
filteredBS = motifsBS.filter("a.id = 'Redwood City Medical Center'")
filteredBS.show(10)

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|{26, Redwood City...|{23, San Mateo Co...|{25, Stanford in ...|
|{26, Redwood City...|{23, San Mateo Co...|{21, Franklin at ...|
|{26, Redwood City...|{24, Redwood City...|{34, Palo Alto Ca...|
|{26, Redwood City...|{23, San Mateo Co...|{26, Redwood City...|
|{26, Redwood City...|{24, Redwood City...|{22, Redwood City...|
|{26, Redwood City...|{24, Redwood City...|{21, Franklin at ...|
|{26, Redwood City...|{24, Redwood City...|{26, Redwood City...|
|{26, Redwood City...|{24, Redwood City...|{24, Redwood City...|
|{26, Redwood City...|{24, Redwood City...|{25, Stanford in ...|
|{26, Redwood City...|{23, San Mateo Co...|{37, Cowper at Un...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [0]:
#store the intermediate computation so they can be reused in subsequent actions.
filteredBS.persist()

Out[49]: DataFrame[a: struct<station_id:int,id:string,lat:double,long:double,dockcount:int,landmark:string,installation:date>, b: struct<station_id:int,id:string,lat:double,long:double,dockcount:int,landmark:string,installation:date>, c: struct<station_id:int,id:string,lat:double,long:double,dockcount:int,landmark:string,installation:date>]

In [0]:
#count the number of connections
filteredBS.count()

Out[50]: 64

In [0]:
# Find stations connected by one trip
motifsBS1 = gbsUnique.find("(a)-[]->(b)").dropDuplicates()
motifsBS1.filter("a.id = 'Redwood City Medical Center'").count()

Out[51]: 7